Compare
- ForecastEvaluator output (research pnl)
- a Portfolio

It can be used:
- In the daily reconciliation flow to compare
    - ForecastEvaluator coming from a simulation
    - portfolio comes from a production system

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
import os

import pandas as pd

import core.config as cconfig
import core.plotting as coplotti
import dataflow.model as dtfmod
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import oms as oms

/app/amp/core/explore.py:39: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import tqdm.autonotebook as tauton


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-db7f3737-6683-48c6-89b2-7886275dcae2.json'
INFO  # Git
  branch_name='AmpTask2534_Prod_reconciliation'
  hash='6c9114caa'
  # Last commits:
    * 6c9114caa saggese  Checkpoint                                                        (38 minutes ago) Wed Aug 31 14:47:57 2022  (HEAD -> AmpTask2534_Prod_reconciliation, origin/AmpTask2534_Prod_reconciliation)
    * 28d80f2c0 saggese  Checkpoint                                                        (55 minutes ago) Wed Aug 31 14:30:11 2022           
    *   203cefa32 saggese  Merge                                                             (64 minutes ago) Wed Aug 31 14:21:35 2022           
    |\  
# Machine info
  system=Linux
  node name=5463b3c4adfd
  release=3.10.0-1160.53.1.el7.x86_64
  version=#1 SMP Fri Jan 14 13:59:45 UTC 2022
  machine=x86_64
  processor=x86_64
  cpu count=8
  cpu freq=scpufreq(current=2499.998, min=

In [22]:
sim_dir = "/app/system_log_dir/forecast_evaluator"
# find /share/data/cf_production/CF_2022_08_29 -name "cf_prod_system_log_dir"
# prod_dir = "/data/cf_production/CF_2022_08_15/job-sasm_job-jobid-1002348952/user_executable_run_0-1000005033091/cf_prod_system_log_dir"
# prod_dir = "/data/cf_production/CF_2022_08_31/job-sasm_job-jobid-1002388639/user_executable_run_0-1000005219664/cf_prod_system_log_dir"
prod_dir = "/data/cf_production/CF_2022_08_29/job-sasm_job-jobid-1002385185/user_executable_run_0-1000005209955/cf_prod_system_log_dir"
prod_dir = os.path.join(prod_dir, "process_forecasts/portfolio")

# Simulation data.
print("# sim_dir")
hdbg.dassert_dir_exists(sim_dir)
!ls {sim_dir}

# Production data.
print("# prod_dir")
hdbg.dassert_dir_exists(prod_dir)
!ls {prod_dir}

# sim_dir
flow  holdings	pnl  position  prediction  price  statistics  volatility
# prod_dir
flows  holdings  holdings_marked_to_market  statistics


In [4]:
date = "2022-08-29"
start_timestamp = pd.Timestamp(date + " 09:30:00", tz="America/New_York")
end_timestamp = pd.Timestamp(date + " 16:00:00", tz="America/New_York")

In [5]:
# hdbg.dassert_dir_exists(root_dir)
dict_ = {
    "portfolio_data_dir": prod_dir,
    "research_data_dir": sim_dir,
    "freq": "15T",
    "portfolio_file_name": None,
    "research_file_name": None,
    "start_timestamp": start_timestamp,
    "end_timestamp": end_timestamp,
}
hdbg.dassert_dir_exists(dict_["portfolio_data_dir"])
hdbg.dassert_dir_exists(dict_["research_data_dir"])

# Load Portfolio data

In [6]:
config = cconfig.Config.from_dict(dict_)
# config = cconfig.get_config_from_nested_dict(dict_)
#
start_timestamp = config["start_timestamp"]
end_timestamp = config["end_timestamp"]

# Load and time-localize Portfolio logged data.
paper_df, paper_stats_df = oms.Portfolio.read_state(
    config["portfolio_data_dir"],
    # file_name=config["portfolio_file_name"],
)
paper_df = paper_df.loc[start_timestamp:end_timestamp]
display(paper_df.head(3))

paper_stats_df = paper_stats_df.loc[start_timestamp:end_timestamp]
display(paper_stats_df.head(3))

Loading `holdings` files...:   0%|          | 0/25 [00:00<?, ?it/s]

Loading `holdings_marked_to_market` files...:   0%|          | 0/25 [00:00<?, ?it/s]

Loading `flows` files...:   0%|          | 0/25 [00:00<?, ?it/s]

Loading `statistics` files...:   0%|          | 0/25 [00:00<?, ?it/s]

holdings                                                                                                                                               holdings_marked_to_market                                                                                                                                                                                                                                                                                  flows                                                                                                                                                                                                                                                     pnl                                                                                                                                                                                                                                                           
                                    10365  11198  12007  12119  13412  13684  14527  14592  14881   15224   15740  15794  16224  16878  16897  17085  17109  17511  17933        -1                         10365         11198         12007         12119         13412         13684         14527         14592         14881         15224         15740         15794         16224         16878         16897         17085         17109         17511         17933        -1            10365        11198        12007        12119        13412        13684        14527        14592        14881        15224        15740        15794        16224        16878        16897        17085        17109        17511        17933         10365         11198         12007         12119         13412         13684         14527         14592         14881         15224         15740         15794         16224         16878        16897         17085         17109         17511         17933
2022-08-30 09:45:14.927222-04:00   -199.0  102.0 -233.0  104.0 -234.0  -98.0 -438.0 -203.0 -377.0 -1090.0 -1223.0 -394.0 -216.0 -194.0  573.0 -312.0 -299.0 -418.0 -376.0      0.000000             -66006.587273  53924.972400 -69565.023667  55492.313760 -22881.835080 -28044.839013 -71210.904320 -54055.358327 -60871.143840 -37247.436400 -56390.760727 -51755.540560 -34705.074240 -30838.678440  81361.798000 -50577.691840 -60892.131387 -56546.064667 -43308.950880      0.000000          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN           NaN          NaN           NaN           NaN           NaN           NaN
2022-08-30 10:00:14.843675-04:00    -16.0   11.0  -14.0    9.0  -20.0   -6.0  -35.0  -19.0  -22.0  -105.0  -111.0  -30.0  -17.0  -15.0   61.0  -11.0  -20.0  -14.0  -19.0  37086.786248              -5295.685000   5813.393644  -4160.110500   4787.433337  -1936.164500  -1704.941700  -5704.700969  -5045.583594  -3526.187500  -3576.427969  -5122.184494  -3916.245188  -2713.798825  -2362.429406   8670.853769  -1775.417325  -4067.868750  -1898.308125  -2185.724375  37086.786248  4980.372394 -5936.679488  3976.170078 -4697.404888  2732.977781  1773.068273  5688.783642  4987.100160  5214.059447  3412.779329  5212.475707  3628.392121  2617.827380  2472.159249 -8401.455509  1779.725973  3753.893305  1941.759305  1950.781988  65691.274667 -54048.258244  69381.083245 -55402.285310  23678.648361  28112.965586  71194.986993  53996.874893  62559.015787  37083.787760  56481.051940  51467.687494  34609.102795  30948.408282 -81092.39974  50582.000488  60578.155942  56589.515847  43074.008493
2022-08-30 10:15:14.692898-04:00    -10.0   11.0  -14.0    9.0  -17.0   -6.0  -35

,pnl,gross_volume,net_volume,gmv,nmv,cash,net_wealth,leverage
2022-08-30 09:45:14.927222-04:00,NaN,0.000000,0.000000,985677.104820,-604118.936500,0.000000,-604118.936500,-1.631594
2022-08-30 10:00:14.843675-04:00,605485.625279,75157.866017,-37086.786248,74263.458969,-35720.097469,37086.786248,1366.688779,54.338237
2022-08-30 10:15:14.692898-04:00,129.230451,14334.306287,436.461181,60022.948138,-35154.405838,36650.325067,1495.919229,40.124458


In [7]:
print(paper_df.index.min(), paper_df.index.max())
display(paper_df.dropna().head(3))
display(paper_df.dropna().tail(3))

2022-08-30 09:45:14.927222-04:00 2022-08-30 15:45:28.516510-04:00


holdings                                                                                                                                             holdings_marked_to_market                                                                                                                                                                                                                                                                flows                                                                                                                                                                                                                                                     pnl                                                                                                                                                                                                                                                            
                                    10365  11198  12007  12119  13412  13684  14527  14592  14881  15224  15740  15794  16224  16878  16897  17085  17109  17511  17933        -1                         10365        11198        12007        12119        13412        13684        14527        14592        14881        15224        15740        15794        16224        16878        16897        17085        17109        17511        17933        -1            10365        11198        12007        12119        13412        13684        14527        14592        14881        15224        15740        15794        16224        16878        16897        17085        17109        17511        17933         10365         11198         12007         12119         13412         13684         14527         14592         14881         15224         15740         15794         16224         16878         16897         17085         17109         17511         17933
2022-08-30 10:00:14.843675-04:00    -16.0   11.0  -14.0    9.0  -20.0   -6.0  -35.0  -19.0  -22.0 -105.0 -111.0  -30.0  -17.0  -15.0   61.0  -11.0  -20.0  -14.0  -19.0  37086.786248              -5295.685000  5813.393644 -4160.110500  4787.433337 -1936.164500 -1704.941700 -5704.700969 -5045.583594 -3526.187500 -3576.427969 -5122.184494 -3916.245188 -2713.798825 -2362.429406  8670.853769 -1775.417325 -4067.868750 -1898.308125 -2185.724375  37086.786248  4980.372394 -5936.679488  3976.170078 -4697.404888  2732.977781  1773.068273  5688.783642  4987.100160  5214.059447  3412.779329  5212.475707  3628.392121  2617.827380  2472.159249 -8401.455509  1779.725973  3753.893305  1941.759305  1950.781988  65691.274667 -54048.258244  69381.083245 -55402.285310  23678.648361  28112.965586  71194.986993  53996.874893  62559.015787  37083.787760  56481.051940  51467.687494  34609.102795  30948.408282 -81092.399740  50582.000488  60578.155942  56589.515847  43074.008493
2022-08-30 10:15:14.692898-04:00    -10.0   11.0  -14.0    9.0  -17.0   -6.0  -35.0  -14.0  -22.0  -60.0 -111.0  -23.0  -16.0  -15.0   13.0  -11.0  -14.0  -15.0  -19.0  36650.325067              -3298.373125  5817.005700 -4141.522175  4770.360000 -1646.226875 -1701.863137 -5695.595938 -3702.818087 -3529.065788 -2035.681500 -5132.882812 -2986.866538 -2541.398900 -2352.365813  1846.905450 -1768.192044 -2842.260488 -2036.972812 -2176.590956  36650.325067 -1972.730898    -0.000000    -0.000000    -0.000000  -289.916392    -0.000000    -0.000000 -1319.678878    -0.000000 -1522.019267    -0.000000  -905.991047  -158.451622    -0.000000  6813.268049    -0.000000 -1216.595630   135.654505    -0.000000     24.580977      3.612056     18.588325    -17.073337      0.021233      3.078563      9.105031     23.086628     -2.878288     18.727202    -10.698319     23.387603     13.948303     10.063594    -10.680270      7.225281      9.012633     -3.010183      9.133419
2022-08-30 10:30:15.161075-04:00    -10.0    9.0  -14.0    6.0  -17.0   -6.0  -35.0  -12.0  -22.0  -60.0 -108.0  -23.0  -16.0  -15.0   -3.0  -11.0  -14.0   1

holdings                                                                                                                                             holdings_marked_to_market                                                                                                                                                                                                                                                        flows                                                                                                                                                                                           pnl                                                                                                                                                                                      
                                    10365  11198  12007  12119  13412  13684  14527  14592  14881  15224  15740  15794  16224  16878  16897  17085  17109  17511  17933        -1                         10365        11198        12007        12119        13412        13684        14527       14592        14881       15224        15740        15794        16224        16878        16897        17085        17109        17511        17933        -1      10365        11198  12007        12119  13412  13684        14527  14592  14881        15224        15740  15794  16224  16878        16897  17085        17109        17511        17933     10365     11198     12007     12119     13412      13684     14527    14592     14881     15224     15740      15794     16224      16878     16897     17085     17109     17511     17933
2022-08-30 15:15:28.633003-04:00     -7.0   -2.0  -10.0   -2.0  -12.0   -5.0   24.0   -8.0  -21.0   23.0  -44.0  -23.0   -9.0  -12.0  -16.0  -11.0    5.0   -9.0    7.0  24021.187097              -2299.343375 -1046.596250 -2929.399188 -1049.423750 -1154.306025 -1381.720781  3906.277500 -2100.67500 -3365.085938  783.088762 -2022.159700 -2954.175344 -1414.999575 -1849.620300 -2251.700000 -1748.267606  1007.678125 -1223.904713   800.572500  24021.187097   -0.0  2617.513396   -0.0    -0.000000   -0.0   -0.0   813.960084   -0.0   -0.0 -1396.360537   275.818457   -0.0   -0.0   -0.0  4363.301805   -0.0    -0.000000   408.010378  -343.626035 -5.915875  3.911209 -8.986688 -3.647562 -0.655350 -11.728781  4.696028 -6.20550 -2.172187 -2.250524 -3.026643 -14.109781 -6.045862 -11.929725  5.311180 -6.335244  2.538719 -0.875972  0.521440
2022-08-30 15:30:28.432593-04:00     -7.0    0.0  -10.0   -2.0  -12.0   -5.0   -8.0   -8.0  -21.0   19.0  -42.0  -23.0   -9.0  -12.0  -19.0  -11.0   -5.0  -27.0    8.0  32996.089822              -2300.958800     0.000000 -2936.306250 -1050.918762 -1156.188450 -1385.644313 -1302.535000 -2103.64245 -3371.817948  647.887294 -1932.433650 -2963.408981 -1416.503025 -1849.377675 -2676.295112 -1749.302775 -1008.290938 -3666.331856   915.594950  32996.089822   -0.0 -1047.955032   -0.0    -0.000000   -0.0   -0.0  5209.935399   -0.0   -0.0   136.379377   -92.032332   -0.0   -0.0   -0.0   422.431228   -0.0  2016.677717  2443.880203  -114.413834 -1.615425 -1.358782 -6.907062 -1.495013 -1.882425  -3.923531  1.122899 -2.96745 -6.732011  1.177908 -2.306282  -9.233637 -1.503450   0.242625 -2.163885 -1.035169  0.708654  1.453059  0.608616
2022-08-30 15:45:28.516510-04:00     -7.0   -3.0  -10.0    1.0  -12.0   -5.0  -15.0   -8.0  -21.0    9.0   -3.0  -23.0   -9.0  -12.0  -28.0  -11.0   -5.0   -6.0   -9.0  33043.790552              -2297.964024 -1570.278375 -2930.637500   524.853769 -1155.661500 -1383.660063 -2439.941156 -2101.96940 -3370.066875  306.584381  -137.848556 -2957.623044 -1414.410412 -1847.472075 -3937.282650 -1746.052412 -1007.984375  -812.807513 -1029.124687  33043.790552   -0.0  1570.517395   -0.0 -1574.731692   -0.0   -0.0  1139.139038   -0.0   -0.0   340.605966 -1792.093054   -0.0   -0.0   -0.0  1265.299078   -0.0    -0.000000 -2844.912509  1943.876508  2.994776  0.239020  5.668750  1.040839  0.526950   1.984250

# Load ForecastEvaluator data

In [8]:
print(config["research_data_dir"])
!ls {config["research_data_dir"]}

/app/system_log_dir/forecast_evaluator
flow  holdings	pnl  position  prediction  price  statistics  volatility


In [9]:
# Load and time localize ForecastEvaluator logged data.
# (
#     research_df,
#     research_stats_df,
# ) = dtfmod.ForecastEvaluatorFromReturns.read_portfolio(
#     config["research_data_dir"],
#     file_name=config["research_file_name"],
# )

(
    research_df,
    research_stats_df,
) = dtfmod.ForecastEvaluatorFromPrices.read_portfolio(
    config["research_data_dir"],
    # file_name=config["research_file_name"],
)

INFO  `file_name`=20220829_170000.csv


In [10]:
print(research_df.index.min(), research_df.index.max())
display(research_df.dropna().head(3))
display(research_df.dropna().tail(3))

2022-08-26 09:15:00-04:00 2022-08-29 17:00:00-04:00


price                                                                                                                                                                                                                       volatility                                                                                                                                                                                     prediction                                                                                                                                                                                        holdings                                                                                                                                                                                                                             position                                                                                                                  \
                                10365       11198       12007       12119       13412       13684       14527       14592       14881      15224      15740       15794       16224       16878       16897       17085       17109       17511       17933      10365     11198     12007     12119     13412     13684     14527     14592     14881     15224     15740     15794     16224     16878     16897     17085     17109     17511     17933      10365     11198     12007     12119     13412     13684     14527     14592     14881     15224     15740     15794     16224     16878     16897     17085     17109     17511     17933       10365       11198       12007      12119       13412      13684       14527       14592       14881        15224       15740       15794       16224       16878       16897       17085       17109       17511       17933         10365         11198         12007         12119         13412         13684         14527         14592         14881   
end_time                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
2022-08-29 10:15:00-04:00  330.530631  527.459832  295.488773  526.619997  101.150208  283.566847  162.611012  265.196549  166.353290  33.800850  46.230158  130.384903  161.419589  160.477207  141.558572  161.853197  202.609134  135.381088  114.011664   0.002982  0.001291  0.003156  0.002190  0.003908  0.006388  0.003015  0.002131  0.004064  0.002589  0.003298  0.004646  0.004170  0.006069  0.001473  0.002211  0.003619  0.001828  0.001865   0.035859 -0.046530 -0.041799  0.008328 -0.044827 -0.135385 -0.023666  0.028398  0.039485  0.057236  0.146152 -0.193321  0.153250  0.021672  0.000687 -0.100420  0.182154  0.103899 -0.058923  -99.391557  -65.437496  -99.604908 -47.753080 -179.066781 -53.429364 -141.131442 -124.659679 -105.465482  -628.538059  512.866717 -182.047047   85.489740  -89.768608  569.015619 -172.733258  132.755018  361.685084 -175.386512 -32851.954120 -34515.650582 -29432.132127 -25147.726601 -18112.642151 -15150.796309 -22949.526700 -33059.316693 -17544.529923   
2022-08-29 10:30:00-04:00  332.208384  528.596186  296.195356  528.43117

price                                                                                                                                                                                                                       volatility                                                                                                                                                                                     prediction                                                                                                                                                                                        holdings                                                                                                                                                                                                                                 position                                                                                                                   \
                                10365       11198       12007       12119       13412       13684       14527       14592       14881      15224      15740       15794       16224       16878       16897       17085       17109       17511       17933      10365     11198     12007     12119     13412     13684     14527     14592     14881     15224     15740     15794     16224     16878     16897     17085     17109     17511     17933      10365     11198     12007     12119     13412     13684     14527     14592     14881     15224     15740     15794     16224     16878     16897     17085     17109     17511     17933       10365       11198       12007       12119        13412       13684       14527       14592       14881        15224        15740       15794       16224       16878       16897       17085       17109       17511       17933         10365         11198         12007         12119          13412         13684         14527         14592         14881   
end_time                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
2022-08-29 15:30:00-04:00  331.397748  531.904317  297.808460  533.853445  100.577734  286.727233  163.547706  266.159273  165.534077  34.017351  46.434595  130.930236  160.333000  159.283058  142.464802  162.213427  203.412283  135.950007  114.438956   0.001618  0.001026  0.001578  0.001755  0.000952  0.003959  0.001426  0.001345  0.001024  0.002304  0.001791  0.002656  0.003366  0.003745  0.001444  0.001654  0.001545  0.002145  0.001693   0.086613 -0.135263 -0.232346  0.411158  0.725156  0.142728  0.377167  0.085877 -0.030364  0.021668 -0.140550 -0.038091  0.149481  0.003238  0.118665 -0.064643 -0.151161  0.057797 -0.110527 -123.047067 -135.280310  169.863559  -99.218958   537.830039 -109.770431  506.335748 -220.724010 -320.986888 -1610.523548 -1289.823740 -341.636808 -248.003302 -268.375566 -532.160300 -278.095578 -251.398490 -483.526558  420.283008 -40777.521063 -71956.180801  50586.805108 -52968.382438   54093.726433 -31474.171884  82810.050039 -58747.742169 -53134.268079   
2022-08-29 15:45:00-04:00  331.173028  532.261895  2

In [11]:
# TODO(gp): @paul add assertion to make sure that there is overlap between research_df and paper_df

## Concat

In [12]:
# # Load and time-localize Portfolio logged data.
# paper_df, paper_stats_df = oms.Portfolio.read_state(
#     config["portfolio_data_dir"],
#     #file_name=config["portfolio_file_name"],
# )

In [13]:
print(start_timestamp, end_timestamp)
paper_df = paper_df.loc[start_timestamp:end_timestamp]
paper_stats_df = paper_stats_df.loc[start_timestamp:end_timestamp]

research_df = research_df.loc[start_timestamp:end_timestamp]
research_stats_df = research_stats_df.loc[start_timestamp:end_timestamp]

# TODO: Check overlap

2022-08-30 09:30:00-04:00 2022-08-30 16:00:00-04:00


In [14]:
print(research_df.columns.levels[0])

# research_df["price"]
# research

Index(['price', 'volatility', 'prediction', 'holdings', 'position', 'flow', 'pnl'], dtype='object')


In [15]:
research_df["position"]

,10365,11198,12007,12119,13412,13684,14527,14592,14881,15224,15740,15794,16224,16878,16897,17085,17109,17511,17933
end_time,,,,,,,,,,,,,,,,,,,


In [16]:
# research_df

In [17]:
research_stats_df

,pnl,gross_volume,net_volume,gmv,nmv
end_time,,,,,


In [18]:
def compute_delay(df: pd.DataFrame, freq: str) -> pd.Series:
    diff = df.index - df.index.round(freq)
    srs = pd.Series(
        [
            diff.mean(),
            diff.std(),
        ],
        [
            "mean",
            "stdev",
        ],
        name="delay",
    )
    return srs


# Compute delay stats.
delay_stats = compute_delay(paper_stats_df, config["freq"])
display(delay_stats)

# Round paper_stats_df to bar
paper_stats_df.index = paper_stats_df.index.round(config["freq"])

mean    0 days 00:00:23.051400720
stdev   0 days 00:00:06.477446663
Name: delay, dtype: timedelta64[ns]

In [19]:
bar_stats_df = pd.concat(
    [research_stats_df, paper_stats_df], axis=1, keys=["research", "paper"]
)
display(bar_stats_df.tail(100))

research                                          paper                                                                                                  
                               pnl gross_volume net_volume gmv nmv            pnl  gross_volume    net_volume            gmv            nmv          cash     net_wealth   leverage
2022-08-30 09:45:00-04:00      NaN          NaN        NaN NaN NaN            NaN      0.000000      0.000000  985677.104820 -604118.936500      0.000000 -604118.936500  -1.631594
2022-08-30 10:00:00-04:00      NaN          NaN        NaN NaN NaN  605485.625279  75157.866017 -37086.786248   74263.458969  -35720.097469  37086.786248    1366.688779  54.338237
2022-08-30 10:15:00-04:00      NaN          NaN        NaN NaN NaN     129.230451  14334.306287    436.461181   60022.948138  -35154.405838  36650.325067    1495.919229  40.124458
2022-08-30 10:30:00-04:00      NaN          NaN        NaN NaN NaN      66.628905   9919.975716    102.797408   55448.480787  -34984.979525  36547.527659    1562.548134  35.485935
2022-08-30 10:45:00-04:00      NaN          NaN        NaN NaN NaN     162.197405   3815.349121   3543.780526   51419.015944  -31279.001594  33003.747133    1724.745539  29.812523
2022-08-30 11:00:00-04:00      NaN          NaN        NaN NaN NaN      73.445159   4259.891279   4259.891279   47009.509294  -26945.665156  28743.855854    1798.190698  26.142672
2022-08-30 11:15:00-04:00      NaN          NaN        NaN NaN NaN     -82.659134   9137.550561   2884.908022   48419.710144  -24143.416269  25858.947833    1715.531564  28.224319
2022-08-30 11:30:00-04:00      NaN          NaN        NaN NaN NaN     -16.816039  12677.829070   -146.802698   45579.635731  -24307.035006  26005.750531    1698.715524  26.831824
2022-08-30 11:45:00-04:00      NaN          NaN        NaN NaN NaN       2.426859   5377.260569   4287.191003   44720.210556  -20017.417144  21718.559527    1701.142384  26.288341
2022-08-30 12:00:00-04:00      NaN          NaN        NaN NaN NaN      17.379735   6864.398754   -656.886851   41470.426785  -20656.924260  22375.446379    1718.522118  24.131448
2022-08-30 12:15:00-04:00      NaN          NaN        NaN NaN NaN     -33.350589  12685.732237 -10060.214269   41430.138719  -30750.489119  32435.660648    1685.171529  24.585117
2022-08-30 12:30:00-04:00      NaN          NaN        NaN NaN NaN      14.497550   8125.891139   6224.583900   38004.714106  -24511.407669  26211.076748    1699.669079  22.360067
2022-08-30 12:45:00-04:00      NaN          NaN        NaN NaN NaN      62.417664   9183.712720  -6691.411367   37478.465066  -31140.401372  32902.488115    1762.086743  21.269364
2022-08-30 13:00:00-04:00      NaN          NaN        NaN NaN NaN      -0.120685   9171.265303   7072.123858   37072.994426  -24068.398199  25830.364256    1761.966058  21.040697
2022-08-30 13:15:00-04:00      NaN          NaN        NaN NaN NaN      12.327253   5223.309188    144.326789   35091.556756  -23911.744156  25686.037467    1774.293311  19.777765
2022-08-30 13:30:00-04:00      NaN          NaN        NaN NaN NaN      53.940545   8206.855249  -1507.866632   34268.784231  -25365.670244  27193.904099    1828.233856  18.744202
2022-08-30 13:45:00-04:00      NaN          NaN        NaN NaN NaN      -7.409041  11700.531894   9165.101247   33315.658362  -16207.978037  18028.802852    1820.824814  18.297015
2022-08-30 14:00:00-04:00      NaN          NaN        NaN NaN NaN     -45.513384   5683.013171   1477.084149   31933.330415  -14776.407272  16551.718703    1775.311431  17.987453
2022-08-30 14:15:00-04:00      NaN          NaN        NaN NaN NaN     -11.805692   8877.312534  -2023.333411   34780.997350  -16811.546375  18575.052114    1763.505739  19.722645
2022-08-30 14:30:00-04:00      NaN          NaN        NaN NaN NaN      -0.691985  16235.003887  -9708.061552   33287.984801  -26520.299912  28283.113666    1762.813754  18.883438
2022-08-30 14:45:00-04:00      NaN          NaN        NaN NaN NaN

In [20]:
stats_computer = dtfmod.StatsComputer()
stats_sxs, _ = stats_computer.compute_portfolio_stats(
    bar_stats_df, config["freq"]
)
display(stats_sxs)

WARNING Empty input series `pnl`


/app/amp/core/statistics/sharpe_ratio.py:156: RuntimeWarning: invalid value encountered in double_scalars
  sr = pnl.mean() / pnl.std()
/app/amp/core/statistics/requires_statsmodels.py:42: RuntimeWarning: invalid value encountered in double_scalars
  kratio = model.params[1] / model.bse[1]


AssertionError: 
################################################################################
* Failed assertion *
0 <= nan
################################################################################


In [ ]:
def per_asset_pnl_corr(
    research_df: pd.DataFrame, paper_df: pd.DataFrame, freq: str
) -> pd.Series:
    research_pnl = research_df["pnl"]
    paper_pnl = paper_df["pnl"]
    corrs = {}
    for asset_id in research_pnl.columns:
        pnl1 = research_pnl[asset_id].resample(freq).sum(min_count=1)
        pnl2 = paper_pnl[asset_id].resample(freq).sum(min_count=1)
        corr = pnl1.corr(pnl2)
        corrs[asset_id] = corr
    corr_srs = pd.Series(corrs).rename("pnl_correlation")
    return corr_srs

In [ ]:
# Display per-asset PnL correlations.
pnl_corrs = per_asset_pnl_corr(research_df, paper_df, config["freq"])
pnl_corrs.hist(bins=101)

In [ ]:
pnl_corrs.sort_values().head()

In [ ]:
pnl = bar_stats_df.T.xs("pnl", level=1).T
display(pnl.head())

In [ ]:
# pnl.corr()
pnl[2:].corr()

In [ ]:
coplotti.plot_portfolio_stats(bar_stats_df[2:])
# coplotti.plot_portfolio_stats(bar_stats_df)